In [1]:
%cd /kaggle/working

/kaggle/working


In [2]:
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns
from hydra import compose, initialize
from omegaconf import OmegaConf

from utils.metric import score

## 提出ファイルを使ってゼロ埋めする

In [3]:
def hide_submission(sub_df):
    hide_sub_df = sub_df.clone().with_columns(
        [pl.lit(0.0).alias(f"ptend_q0002_{i}") for i in range(12, 22)]
    )
    return hide_sub_df

In [4]:
base_sub_path = "gs://kaggle-leap/kami/20240610_ensemble_nelder_mead2_lb078544.parquet"
base_sub_df = pl.read_parquet(base_sub_path)

In [5]:
base_sub_df.head()

sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_0""",0.000003,-0.000065,-0.000085,-0.000086,-0.000096,-0.0001,-0.000082,-0.000064,-0.000049,-0.000041,-0.000028,-0.000018,-0.000009,-0.000004,-4.1657e-7,0.000001,0.000006,0.000004,0.000003,-0.000009,0.000009,-0.000003,-0.000012,-0.000016,-0.000015,-0.000013,-0.000016,-0.000018,-0.000023,-0.000031,-0.000037,-0.000032,-0.000023,-0.000018,-0.000017,-0.000015,…,9.3603e-7,0.000002,-4.3578e-7,-0.000002,-0.000001,-6.4969e-7,-8.2227e-7,-0.000003,-7.8813e-7,-5.8951e-7,1.9074e-7,-0.000003,0.000006,0.000001,0.000002,-0.000005,-0.000006,-0.000026,-0.000048,-0.000048,-0.000018,0.000038,0.000043,0.000042,0.000022,0.000012,0.000009,0.000008,4.1996e-7,-0.136167,402.35946,4.6684e-11,4.1327e-10,-0.033577,-0.049489,0.09329,0.212398
"""test_10""",-0.000015,-0.00005,-0.000034,-0.000052,-0.00009,-0.000133,-0.000142,-0.000097,-0.000046,-0.000026,-0.000018,-0.000014,-0.000011,-0.000009,-0.000007,-0.000005,-0.000004,-0.000004,-0.000002,-0.000004,-0.000004,-0.000008,-0.000008,-0.000009,-0.000008,-0.000011,-0.000013,-0.000017,-0.00002,-0.000018,-0.00002,-0.000019,-0.000015,-0.000007,-0.000002,0.000003,…,-4.8681e-7,-3.7356e-7,2.0193e-7,4.7321e-7,3.4525e-7,2.1168e-7,1.7050e-7,6.5687e-8,2.0065e-8,2.7567e-9,1.3472e-7,-8.9901e-7,4.5322e-7,-3.2823e-7,0.000001,-4.6198e-9,-5.7225e-7,-0.000002,-0.000002,-0.000002,-0.000002,-0.000001,-1.4282e-7,9.2288e-7,0.000002,0.000001,8.3161e-7,-0.000002,0.000004,-0.027761,345.154827,2.5575e-11,-5.9339e-10,-0.316943,-0.456919,0.0302,0.172592
"""test_100""",-0.000007,-0.00008,-0.000033,-0.000028,-0.000073,-0.000096,-0.000079,-0.000053,-0.00003,-0.000021,-0.000017,-0.000014,-0.000012,-0.00001,-0.000007,-0.000005,-0.000004,-0.000005,0.000004,-0.000009,-0.000004,-0.000007,-0.000006,-0.000006,-0.000004,-0.000003,-0.000006,-0.000006,-0.000007,-0.000011,-0.000015,-0.000019,-0.000022,-0.000022,-0.000024,-0.000024,…,-0.000002,-0.000002,-7.8204e-7,2.2820e-7,8.2554e-7,0.000001,8.8598e-7,8.5306e-7,6.2661e-7,3.6903e-7,1.7289e-8,-4.6920e-7,-1.9907e-7,0.000002,3.3318e-7,-0.000001,-7.2683e-7,-0.000004,-0.00001,-0.00002,-0.000023,0.000009,0.00001,0.000014,0.00001,0.000004,0.000004,0.000004,0.000012,2.065205,331.870707,3.9388e-12,6.2842e-10,0.947471,0.65194,0.288081,0.205645
"""test_1000""",0.000026,0.000007,0.000033,0.000026,-0.000013,-0.000057,-0.000047,-0.000021,-0.000008,-0.000003,-0.000001,-6.6297e-7,-3.7065e-7,0.000002,0.000003,0.000005,0.000006,0.000004,1.3132e-7,7.6130e-7,-4.3587e-7,-0.000002,-0.00001,-0.000019,-0.000017,-0.000014,-0.000014,-0.000017,-0.000021,-0.000023,-0.000021,-0.00002,-0.000018,-0.000017,-0.000016,-0.000017,…,2.6728e-8,-4.6045e-8,6.6996e-7,-0.000001,8.5986e-8,0.000001,-0.000001,1.5021e-7,-9.4691e-7,3.3675e-7,-0.000005,0.000007,0.000003,0.000005,0.000007,0.000007,0.000007,0.00001,0.00001,0.000008,0.000006,0.000004,9.9584e-7,-0.000006,-0.000012,-0.000022,-0.000039,-0.000046,0.000025,2.73912,361.004821,-3.

In [6]:
# hide
hide_base_sub_df = hide_submission(base_sub_df)
print(hide_base_sub_df.shape)
hide_base_sub_df[:, 120:150].head()

(625000, 369)


ptend_q0001_59,ptend_q0002_0,ptend_q0002_1,ptend_q0002_2,ptend_q0002_3,ptend_q0002_4,ptend_q0002_5,ptend_q0002_6,ptend_q0002_7,ptend_q0002_8,ptend_q0002_9,ptend_q0002_10,ptend_q0002_11,ptend_q0002_12,ptend_q0002_13,ptend_q0002_14,ptend_q0002_15,ptend_q0002_16,ptend_q0002_17,ptend_q0002_18,ptend_q0002_19,ptend_q0002_20,ptend_q0002_21,ptend_q0002_22,ptend_q0002_23,ptend_q0002_24,ptend_q0002_25,ptend_q0002_26,ptend_q0002_27,ptend_q0002_28
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-3.6495e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5384e-52,-1.1441e-50,-3.8119e-46,0.0,0.0,-3.1695e-14
2.8797e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.7835e-38,-1.1512e-34,0.0,-1.5774e-13
-1.6525e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.6324e-55,-4.9586e-50,-1.7488e-45,-3.0572e-41,-1.6994e-36,-4.7132e-32,-5.3210e-14
-7.1831e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.2260e-33,-1.4761e-28,1.1855e-13
-6.1449e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.3233e-46,-1.0059e-41,-4.6097e-37,-1.4812e-32,-4.3854e-28,-1.1530e-23,-3.5388e-14


In [7]:
# hide_base_sub_df.write_parquet("output/hide_lb078544_sub.parquet")

### location使った予測で20%を埋める

In [6]:
# multi-column to single-column の読み込み

multi_col_sub_path = (
    "gs://kaggle-leap/kami/experiments/502_latlon_smoothl1_vast/all_001/submission.parquet"
)


"""
multi_col_sub_path = (
    "gs://kaggle-leap/kami/20240610_ensemble_nelder_mead2_lb080254.parquet"
)

multi_col_sub_path = (
    "gs://kaggle-leap/kami/experiments/500_latlon_gnn/all/submission.parquet"
)
"""

multi_col_sub_df = pl.read_parquet(multi_col_sub_path)

# 重みが変わったので新しいスケールに変換する
ss_old_df = pl.read_csv(
    "input/leap-atmospheric-physics-ai-climsim/sample_submission_old.csv", n_rows=1
)
weight_old_array = ss_old_df.select(
    [x for x in ss_old_df.columns if x != "sample_id"]
).to_numpy()[0]

ss_df = pl.read_csv(
    "input/leap-atmospheric-physics-ai-climsim/sample_submission.csv", n_rows=1
)
weight_array = ss_df.select([x for x in ss_df.columns if x != "sample_id"]).to_numpy()[
    0
]

In [7]:
from sklearn.metrics import r2_score
from tqdm import tqdm

multi_col_pred = multi_col_sub_df[:, 1:].to_numpy()
multi_col_pred = multi_col_pred / weight_old_array  # old array は一部０なので注意。
multi_col_pred[np.isnan(multi_col_pred)] = 0
multi_col_pred *= weight_array

multi_col_sub_df = pl.concat(
    [
        multi_col_sub_df.select("sample_id"),
        pl.from_numpy(multi_col_pred, schema=multi_col_sub_df.columns[1:]),
    ],
    how="horizontal",
)
multi_col_sub_df.head()

/tmp/ipykernel_355/2850579238.py:5: RuntimeWarning: invalid value encountered in divide
  multi_col_pred = multi_col_pred / weight_old_array  # old array は一部０なので注意。


sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_169651""",-0.000009,-0.000026,-0.000023,-0.000045,-0.000078,-0.000104,-0.0001,-0.000081,-0.000052,-0.000039,-0.000028,-0.000021,-0.000016,-0.000009,-0.000002,0.000002,0.000003,8.6960e-7,0.000003,-8.0291e-7,-0.000003,-0.000006,-0.00001,-0.000012,-0.000013,-0.000015,-0.000019,-0.000023,-0.000026,-0.000031,-0.000033,-0.000032,-0.000031,-0.000027,-0.000026,-0.000026,…,5.2833e-8,-7.5532e-9,9.3756e-8,-5.8854e-7,3.7382e-7,9.7996e-9,2.5200e-8,4.9315e-8,-6.5331e-8,1.4989e-7,-3.5992e-7,2.9804e-7,-3.0610e-7,4.5318e-7,-7.3513e-8,7.5902e-9,3.8205e-9,9.3413e-9,1.9214e-8,-1.4920e-8,-4.7933e-8,3.5813e-8,-2.4742e-8,2.6293e-7,-5.2895e-7,2.5100e-7,-1.2951e-7,-0.000008,0.000008,2.480143,385.920439,-3.0178e-11,6.8789e-10,0.395578,-0.00919,-0.239849,-0.129868
"""test_524862""",-0.000007,-0.00002,-0.000024,-0.000045,-0.000082,-0.000108,-0.000101,-0.000078,-0.00005,-0.000039,-0.00003,-0.000023,-0.000018,-0.000013,-0.000006,-0.000001,9.3235e-7,6.4516e-7,-0.000001,-0.000001,-0.000004,-0.000006,-0.000008,-0.000009,-0.00001,-0.000013,-0.000018,-0.000023,-0.000024,-0.000024,-0.000024,-0.000024,-0.000024,-0.000025,-0.000025,-0.000024,…,-1.9001e-7,-9.1313e-8,1.3577e-7,2.3358e-7,2.4246e-7,3.3895e-7,2.4236e-7,2.0554e-8,-1.0593e-7,-1.4625e-7,-9.7939e-8,4.4453e-8,-1.3246e-7,6.3841e-7,5.6296e-8,-2.7759e-7,-2.5758e-7,-2.6453e-7,-2.6488e-7,-5.2587e-7,-5.3528e-7,4.6001e-9,-2.7778e-7,-6.9526e-7,-6.7627e-7,-0.000003,-0.000005,-0.00001,0.000021,0.132491,377.073801,-1.4507e-14,9.4971e-9,0.092854,0.071274,0.506678,0.09462
"""test_634129""",-0.00001,-0.000048,-0.000036,-0.000047,-0.000073,-0.000102,-0.000104,-0.000082,-0.000048,-0.000035,-0.000025,-0.000018,-0.000012,-0.000008,-0.000003,0.000003,0.000006,0.000004,0.000002,-0.000003,-0.000001,-0.000005,-0.000007,-0.000019,-0.000026,-0.000034,-0.000045,-0.000045,-0.000041,-0.000027,-0.000036,-0.000035,-0.000032,-0.000026,-0.000023,-0.000027,…,-0.000003,4.2070e-7,-2.2409e-7,-9.5184e-7,-9.2669e-7,-2.7099e-7,9.4895e-8,2.3829e-7,2.3587e-7,-6.4265e-8,-2.0723e-7,-2.8360e-7,-1.0951e-7,5.1781e-8,-9.6769e-8,-8.9239e-7,-1.8043e-8,2.4545e-8,-4.9017e-7,-6.1515e-7,1.6870e-7,2.0200e-7,4.1661e-7,3.7738e-7,0.000001,0.000002,-3.3106e-7,7.4349e-7,-0.000002,-0.561012,411.929581,-1.3452e-10,-8.8280e-11,-0.794015,-0.900364,0.036745,0.142744
"""test_403572""",-0.000011,-0.000038,-0.00003,-0.000043,-0.000072,-0.000105,-0.000106,-0.000081,-0.000047,-0.000035,-0.000026,-0.000019,-0.000013,-0.000009,-0.000004,0.000001,0.000004,0.000003,0.000004,-0.000003,-3.2584e-7,-0.000006,-0.000009,-0.000021,-0.000027,-0.000023,-0.000019,-0.000025,-0.000033,-0.00003,-0.000028,-0.000025,-0.000025,-0.000031,-0.000031,-0.000034,…,-3.6149e-7,-5.4129e-7,-1.6777e-7,-1.1677e-8,4.3131e-7,4.1013e-7,3.0330e-7,7.0967e-8,-1.4785e-7,-2.1640e-7,-8.1225e-7,-0.000003,8.0439e-7,0.000002,-2.8376e-7,2.2105e-7,7.3670e-8,-4.7792e-7,-0.000001,-0.000003,-0.000004,-0.000003

In [8]:
# 重複範囲を取得
test_df = pl.read_csv("input/leap-atmospheric-physics-ai-climsim/test.csv")
test_old_df = pl.read_csv("input/leap-atmospheric-physics-ai-climsim/test_old.csv")
tmp = test_df[:, 1::10].with_row_index("index")
tmp_old = test_old_df[:, 1::10].with_row_index("old_index")
merge_df = pd.merge(tmp.to_pandas().round(5), tmp_old.to_pandas().round(5))
# 新しいテストでの重複 index
duplicated_index = merge_df["index"]

# 古いテストでの重複 index
duplicated_old_index = merge_df["old_index"]

In [9]:
(merge_df["old_index"] // 384).value_counts().value_counts()

count
384    326
Name: count, dtype: int64

In [10]:
# 予測値を置き換える(但し、旧版weightで０のやつは置き換えない）

sub_pred = base_sub_df[:, 1:].to_numpy()
multi_col_sub_pred = multi_col_sub_df[:, 1:].to_numpy()

for ci, w in enumerate(weight_old_array):
    if w != 0:
        sub_pred[duplicated_index, ci] = multi_col_sub_pred[duplicated_old_index, ci]

fill_sub_df = pl.concat(
    [
        test_df.select("sample_id"),
        pl.from_numpy(sub_pred, schema=base_sub_df.columns[1:]),
    ],
    how="horizontal",
)
fill_sub_df.head(10)

sample_id,ptend_t_0,ptend_t_1,ptend_t_2,ptend_t_3,ptend_t_4,ptend_t_5,ptend_t_6,ptend_t_7,ptend_t_8,ptend_t_9,ptend_t_10,ptend_t_11,ptend_t_12,ptend_t_13,ptend_t_14,ptend_t_15,ptend_t_16,ptend_t_17,ptend_t_18,ptend_t_19,ptend_t_20,ptend_t_21,ptend_t_22,ptend_t_23,ptend_t_24,ptend_t_25,ptend_t_26,ptend_t_27,ptend_t_28,ptend_t_29,ptend_t_30,ptend_t_31,ptend_t_32,ptend_t_33,ptend_t_34,ptend_t_35,…,ptend_v_31,ptend_v_32,ptend_v_33,ptend_v_34,ptend_v_35,ptend_v_36,ptend_v_37,ptend_v_38,ptend_v_39,ptend_v_40,ptend_v_41,ptend_v_42,ptend_v_43,ptend_v_44,ptend_v_45,ptend_v_46,ptend_v_47,ptend_v_48,ptend_v_49,ptend_v_50,ptend_v_51,ptend_v_52,ptend_v_53,ptend_v_54,ptend_v_55,ptend_v_56,ptend_v_57,ptend_v_58,ptend_v_59,cam_out_NETSW,cam_out_FLWDS,cam_out_PRECSC,cam_out_PRECC,cam_out_SOLS,cam_out_SOLL,cam_out_SOLSD,cam_out_SOLLD
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""test_0""",0.000003,-0.000065,-0.000085,-0.000086,-0.000096,-0.0001,-0.000082,-0.000064,-0.000049,-0.000041,-0.000028,-0.000018,-0.000009,-0.000004,-4.1657e-7,0.000001,0.000006,0.000004,0.000003,-0.000009,0.000009,-0.000003,-0.000012,-0.000016,-0.000015,-0.000013,-0.000016,-0.000018,-0.000023,-0.000031,-0.000037,-0.000032,-0.000023,-0.000018,-0.000017,-0.000015,…,9.3603e-7,0.000002,-4.3578e-7,-0.000002,-0.000001,-6.4969e-7,-8.2227e-7,-0.000003,-7.8813e-7,-5.8951e-7,1.9074e-7,-0.000003,0.000006,0.000001,0.000002,-0.000005,-0.000006,-0.000026,-0.000048,-0.000048,-0.000018,0.000038,0.000043,0.000042,0.000022,0.000012,0.000009,0.000008,4.1996e-7,-0.136167,402.35946,4.6684e-11,4.1327e-10,-0.033577,-0.049489,0.09329,0.212398
"""test_10""",-0.000015,-0.00005,-0.000034,-0.000052,-0.00009,-0.000133,-0.000142,-0.000097,-0.000046,-0.000026,-0.000018,-0.000014,-0.000011,-0.000009,-0.000007,-0.000005,-0.000004,-0.000004,-0.000002,-0.000004,-0.000004,-0.000008,-0.000008,-0.000009,-0.000008,-0.000011,-0.000013,-0.000017,-0.00002,-0.000018,-0.00002,-0.000019,-0.000015,-0.000007,-0.000002,0.000003,…,-4.8681e-7,-3.7356e-7,2.0193e-7,4.7321e-7,3.4525e-7,2.1168e-7,1.7050e-7,6.5687e-8,2.0065e-8,2.7567e-9,1.3472e-7,-8.9901e-7,4.5322e-7,-3.2823e-7,0.000001,-4.6198e-9,-5.7225e-7,-0.000002,-0.000002,-0.000002,-0.000002,-0.000001,-1.4282e-7,9.2288e-7,0.000002,0.000001,8.3161e-7,-0.000002,0.000004,-0.027761,345.154827,2.5575e-11,-5.9339e-10,-0.316943,-0.456919,0.0302,0.172592
"""test_100""",-0.000007,-0.00008,-0.000033,-0.000028,-0.000073,-0.000096,-0.000079,-0.000053,-0.00003,-0.000021,-0.000017,-0.000014,-0.000012,-0.00001,-0.000007,-0.000005,-0.000004,-0.000005,0.000004,-0.000009,-0.000004,-0.000007,-0.000006,-0.000006,-0.000004,-0.000003,-0.000006,-0.000006,-0.000007,-0.000011,-0.000015,-0.000019,-0.000022,-0.000022,-0.000024,-0.000024,…,-0.000002,-0.000002,-7.8204e-7,2.2820e-7,8.2554e-7,0.000001,8.8598e-7,8.5306e-7,6.2661e-7,3.6903e-7,1.7289e-8,-4.6920e-7,-1.9907e-7,0.000002,3.3318e-7,-0.000001,-7.2683e-7,-0.000004,-0.00001,-0.00002,-0.000023,0.000009,0.00001,0.000014,0.00001,0.000004,0.000004,0.000004,0.000012,2.065205,331.870707,3.9388e-12,6.2842e-10,0.947471,0.65194,0.288081,0.205645
"""test_1000""",0.000026,0.000007,0.000033,0.000026,-0.000013,-0.000057,-0.000047,-0.000021,-0.000008,-0.000003,-0.000001,-6.6297e-7,-3.7065e-7,0.000002,0.000003,0.000005,0.000006,0.000004,1.3132e-7,7.6130e-7,-4.3587e-7,-0.000002,-0.00001,-0.000019,-0.000017,-0.000014,-0.000014,-0.000017,-0.000021,-0.000023,-0.000021,-0.00002,-0.000018,-0.000017,-0.000016,-0.000017,…,2.6728e-8,-4.6045e-8,6.6996e-7,-0.000001,8.5986e-8,0.000001,-0.000001,1.5021e-7,-9.4691e-7,3.3675e-7,-0.000005,0.000007,0.000003,0.000005,0.000007,0.000007,0.000007,0.00001,0.00001,0.000008,0.000006,0.000004,9.9584e-7,-0.000006,-0.000012,-0.000022,-0.000039,-0.000046,0.000025,2.73912,361.004821,-3.

In [11]:
# 最終的に提出したいもの
fill_sub_df.write_parquet("output/fill_lb078544_fill_lb08073_sub.parquet")

In [12]:
# 埋める前後でスコアが大きく変わらないかをチェック
preds = base_sub_df[:, 1:].to_numpy()
labels = fill_sub_df[:, 1:].to_numpy()
_predict_df = pd.DataFrame(
    preds * weight_array, columns=[i for i in range(preds.shape[1])]
).reset_index()
_label_df = pd.DataFrame(
    labels * weight_array, columns=[i for i in range(labels.shape[1])]
).reset_index()
r2_scores = score(_label_df, _predict_df, "index", multioutput="raw_values")

r2_score = float(np.array(r2_scores).mean())
print(f"{r2_score=}")

r2_score=0.9896371489368974


In [16]:
# hide
hide_fill_sub_df = hide_submission(fill_sub_df)
print(hide_fill_sub_df.shape)
hide_fill_sub_df.write_parquet(
    "output/hide_lb078544_fill_500_latlon_gnn_all_sub.parquet"
)
hide_fill_sub_df[:, 120:150].head()

(625000, 369)


ptend_q0001_59,ptend_q0002_0,ptend_q0002_1,ptend_q0002_2,ptend_q0002_3,ptend_q0002_4,ptend_q0002_5,ptend_q0002_6,ptend_q0002_7,ptend_q0002_8,ptend_q0002_9,ptend_q0002_10,ptend_q0002_11,ptend_q0002_12,ptend_q0002_13,ptend_q0002_14,ptend_q0002_15,ptend_q0002_16,ptend_q0002_17,ptend_q0002_18,ptend_q0002_19,ptend_q0002_20,ptend_q0002_21,ptend_q0002_22,ptend_q0002_23,ptend_q0002_24,ptend_q0002_25,ptend_q0002_26,ptend_q0002_27,ptend_q0002_28
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-3.6495e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.5384e-52,-1.1441e-50,-3.8119e-46,0.0,0.0,-3.1695e-14
2.8797e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.7835e-38,-1.1512e-34,0.0,-1.5774e-13
-1.6525e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-4.6324e-55,-4.9586e-50,-1.7488e-45,-3.0572e-41,-1.6994e-36,-4.7132e-32,-5.3210e-14
-7.1831e-8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.2260e-33,-1.4761e-28,1.1855e-13
-6.1449e-10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.3233e-46,-1.0059e-41,-4.6097e-37,-1.4812e-32,-4.3854e-28,-1.1530e-23,-3.5388e-14


## validationから定数埋めでどれだけスコアが下がりそうか確認

In [ ]:
config_dir = "../experiments"
exp_name = "201_unet_multi/all"

In [4]:
with initialize(
    version_base=None, config_path=f"{config_dir}/{exp_name.split('/')[0]}"
):
    cfg = compose(
        config_name="config.yaml",
        overrides=[f"exp={exp_name.split('/')[-1]}"],
        return_hydra_config=True,
    )

# 定数定義
output_dir = Path(f"output/experiments/{exp_name}")
gcs_path = f"gs://{cfg.dir.gcs_bucket}/{cfg.dir.gcs_base_dir}/experiments/{exp_name}/"

# 結果などの読み込み
r2_score_dict = pickle.load(open(output_dir / "val2_r2_score_dict.pkl", "rb"))
print("r2: ", np.mean(list(r2_score_dict.values())))

/kaggle/working
r2:  0.7815333871933391


In [5]:
predict_df = pl.read_parquet(gcs_path + "val2_predict.parquet", retries=5)
label_df = pl.read_parquet(gcs_path + "val2_label.parquet", retries=5)
ss_df = pl.read_csv(
    "input/leap-atmospheric-physics-ai-climsim/sample_submission.csv", n_rows=1
)
weight_array = ss_df.select([x for x in ss_df.columns if x != "sample_id"]).to_numpy()[
    0
]

In [41]:
# 通常
preds = predict_df[:, 1:].to_numpy()
labels = label_df[:, 1:].to_numpy()
_predict_df = pd.DataFrame(
    preds * weight_array, columns=[i for i in range(preds.shape[1])]
).reset_index()
_label_df = pd.DataFrame(
    labels * weight_array, columns=[i for i in range(labels.shape[1])]
).reset_index()
r2_scores = score(_label_df, _predict_df, "index", multioutput="raw_values")

r2_score = float(np.array(r2_scores).mean())
print(f"{r2_score=}")

r2_score=0.7815333871933391


In [42]:
r2_scores[132:142]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [44]:
# 10個の後処理結果を0埋めに変換

preds = predict_df[:, 1:].to_numpy()
preds[:, 132:142] = 0
labels = label_df[:, 1:].to_numpy()
_predict_df = pd.DataFrame(
    preds * weight_array, columns=[i for i in range(preds.shape[1])]
).reset_index()
_label_df = pd.DataFrame(
    labels * weight_array, columns=[i for i in range(labels.shape[1])]
).reset_index()
r2_scores = score(_label_df, _predict_df, "index", multioutput="raw_values")

r2_score = float(np.array(r2_scores).mean())
print(f"{r2_score=}")

r2_score=0.7538482690839238


In [47]:
(1 - r2_scores[132:142]).sum() / 368

0.027685118109415333

In [48]:
(1 - r2_scores[132:142]).sum() / 368 + 0.7538482690839238

0.7815333871933391